<img src="https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png" style="float:  left; margin: 10px;">

# K Nearest Neighbors classification Lab 
---
Week 4 | Lab 4.1


# KNN for Classification 

In this lab you'll practice using KNN for classification, get exposure to Gridsearch will is a version of crossvalidation that will select a model's optimual parameters for you. 

---

### 1. Load packages

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from sklearn.neighbors import KNeighborsClassifier

---

### 2. Load datasets


In [9]:
affair = pd.read_csv('~/DSI-SF-5/datasets/affairs/affair.csv')
churn = pd.read_csv('~/DSI-SF-5/datasets/cell_phone_churn/cell_phone_churn.csv')

---

### 3. Encode affairs vs. not in affair dataset 

Look at the nbaffairs feature. You'll see that values are the counts of arrairs that people have had in the past. 

Write a function that will turn the values in nbaffairs feature into binary categories: zero for no affairs and one if they've had an affair. 

This will be your binary target class variable.

In [6]:
affair.head()

,Unnamed: 0,sex,age,ym,child,religious,education,occupation,rate,nbaffairs
0,1,male,37.0,10.00,no,3,18,7,4,0
1,2,female,27.0,4.00,no,4,14,6,4,0
2,3,female,32.0,15.00,yes,1,12,1,4,0
3,4,male,57.0,15.00,yes,5,18,6,5,0
4,5,male,22.0,0.75,no,2,17,6,3,0


In [7]:
def binary_affair(x):
    if x == 0:
        return 0
    else:
        return 1
    
affair['had_affair'] = affair.nbaffairs.map(binary_affair)

y = affair.had_affair.values

---

### 4. Clean and convert string variables

1. Drop any uncessary features
2. Transform string values into numerical values (binarize them)

In [11]:
affair.head()

,Unnamed: 0,sex,age,ym,child,religious,education,occupation,rate,nbaffairs
0,1,male,37.0,10.00,no,3,18,7,4,0
1,2,female,27.0,4.00,no,4,14,6,4,0
2,3,female,32.0,15.00,yes,1,12,1,4,0
3,4,male,57.0,15.00,yes,5,18,6,5,0
4,5,male,22.0,0.75,no,2,17,6,3,0


In [12]:
affair.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
affair.sex.unique()

array(['male', 'female'], dtype=object)

In [14]:
affair.child.unique()

array(['no', 'yes'], dtype=object)

In [15]:
affair.sex = affair.sex.map(lambda x: 1 if x == 'male' else 0)
affair.child = affair.child.map(lambda x: 1 if x == 'yes' else 0)

---

### 5. Fit a `KNeighborsClassifier` with `weights='uniform'` and `n_neighbors=3`

You should choose **2 predictor variables** to predict wheather someone has had affair or not

In [18]:
np.mean(y)

0.24958402662229617

In [19]:
X = affair[['age','religious']].values

knn_uni_n3 = KNeighborsClassifier(n_neighbors=3, weights='uniform')
knn_uni_n3.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

---

### 6. Cross-validate the classifier with `StratifiedKFold`

Write a function that implements cross validation. This function will score models based on the accuracy metic, and StratifiedKFold to create folds. 

StratifiedKFold will create K folds of your data. The "Stratified" part in " StratifiedKFold" will meake sure that each fold has an equal number of each type of classifications (i.e. balanced classes). Training a model on unbalanced classes will bias its performance; the model will get really good at predicitng the majority class and preform poorly on predicting the minority. 

Read the [StratifiedKFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) documentation to learn how to use this built-in function. There is an example at the bottom of the page in the docs. 

In [36]:
from sklearn.model_selection import StratifiedKFold

# Function to crossvalidate accuracy of a knn model acros folds
def accuracy_crossvalidator(X, y, knn):
    
    scores = []
    cv_indices = StratifiedKFold(n_splits=5)
    for train_i, test_i in cv_indices.split(X,y):
        
        X_train = X[train_i, :]
        X_test = X[test_i, :]

        y_train = y[train_i]
        y_test = y[test_i]

        knn.fit(X_train, y_train)
        
        acc = knn.score(X_test, y_test)
        scores.append(acc)
        
        
    return scores, np.mean(scores)
    

In [38]:
accs, mean_acc = accuracy_crossvalidator(X, y, knn_uni_n3)
accs, mean_acc

([0.73553719008264462,
  0.71666666666666667,
  0.7416666666666667,
  0.70833333333333337,
  0.69999999999999996],
 0.7204407713498624)

---

### 7. Do the same but with  `n_neighbors=11`

Use the same predictor variables and cv folds.

In [39]:
knn_uni_n11 = KNeighborsClassifier(n_neighbors=11, weights='uniform')
knn_uni_n11.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=11, p=2,
           weights='uniform')

In [40]:
accs, mean_acc = accuracy_crossvalidator(X, y, knn_uni_n11)
accs, mean_acc

([0.72727272727272729, 0.73333333333333328, 0.7416666666666667, 0.75, 0.75],
 0.74045454545454548)

---

### 8. Cross-validate a model with  `n_neighbors=11` and `weights='distance'`

In [41]:
knn_dist_n11 = KNeighborsClassifier(n_neighbors=11, weights='distance')
knn_dist_n11.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=11, p=2,
           weights='distance')

In [43]:
accs, mean_acc = accuracy_crossvalidator(X, y, knn_dist_n11)
accs, mean_acc

([0.71900826446280997, 0.72499999999999998, 0.75, 0.7416666666666667, 0.75],
 0.73713498622589535)

---

### 9. With the churn dataset, find the optimal neighbors and weighting to predict churn using gridsearch

Show the cross-validated accuracy of the model.

In [47]:
from sklearn.model_selection import GridSearchCV

In [48]:
churn.head()

,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,eve_mins,eve_calls,eve_charge,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,churn
0,KS,128,415,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,yes,no,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,yes,no,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [49]:
churn.churn.unique()

array([False, True], dtype=object)

In [50]:
churn.churn = churn.churn.map(lambda x: 1 if x else 0)

In [54]:
churn.intl_plan = churn.intl_plan.map(lambda x: 1 if x == 'yes' else 0)
churn.vmail_plan = churn.vmail_plan.map(lambda x: 1 if x == 'yes' else 0)

In [55]:
y = churn.churn.values
X = churn[[col for col in churn.columns if not col in ['state','churn']]].values

In [59]:
params = {
    'n_neighbors':range(1,20),
    'weights':['uniform','distance']
}

knn = KNeighborsClassifier()
knn_gs = GridSearchCV(knn, params, cv=4, verbose=1, n_jobs=3)

knn_gs.fit(X, y)

Fitting 4 folds for each of 38 candidates, totalling 152 fits


[Parallel(n_jobs=3)]: Done 152 out of 152 | elapsed:   10.2s finished


GridSearchCV(cv=4, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 'weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [60]:
print knn_gs.best_params_
best_knn = knn_gs.best_estimator_

{'n_neighbors': 13, 'weights': 'distance'}


In [66]:
#cv_indices = StratifiedKFold(n_splits=5)

accs, mean_accs = accuracy_crossvalidator(X, y, best_knn)
accs, mean_accs

([0.88455772113943032,
  0.8710644677661169,
  0.88005997001499248,
  0.87687687687687688,
  0.88288288288288286],
 0.87908838373605991)

---

## Variable imputation with KNeighbors

You can actually do both classification _and_ regression with KNN. It is quite flexible due to its simplicity. One of it's most useful features is the ability to perform very nice imputation.

---

### 10. Look at the coffee data, count the missing values

In [67]:
coffee.head()

,Timestamp,Name,Starbucks,PhilzCoffee,BlueBottleCoffee,PeetsTea,CaffeTrieste,GrandCoffee,RitualCoffee,FourBarrel,WorkshopCafe
0,3/17/2015 18:37:58,Alison,3,5,4.0,3.0,NaN,NaN,5.0,5.0,NaN
1,3/17/2015 18:38:09,April,4,5,5.0,3.0,NaN,NaN,3.0,NaN,5.0
2,3/17/2015 18:38:25,Vijay,3,5,5.0,5.0,3.0,2.0,1.0,1.0,1.0
3,3/17/2015 18:38:28,Vanessa,1,5,5.0,2.0,NaN,NaN,3.0,2.0,3.0
4,3/17/2015 18:38:46,Isabel,1,4,4.0,2.0,4.0,NaN,4.0,4.0,NaN


In [68]:
coffee.isnull().sum()

Timestamp            0
Name                 0
Starbucks            0
PhilzCoffee          0
BlueBottleCoffee     6
PeetsTea             1
CaffeTrieste        12
GrandCoffee         15
RitualCoffee         7
FourBarrel          10
WorkshopCafe         8
dtype: int64